In [1]:
%%capture
!pip install transformers==4.31 # version specific 

In [15]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 6.6 MB/s eta 0:00:000:00:0100:01


In [1]:
from lavis.models import model_zoo

import re
import torch
from torch import nn
from torchvision import transforms

import json

from PIL import Image

import cv2
import numpy as np

from skimage import transform as skimage_transform
from scipy.ndimage import filters
from matplotlib import pyplot as plt

/opt/conda/envs/pytorch/lib/python3.10/site-packages/diffusers/models/cross_attention.py:30: FutureWarning: Importing from cross_attention is deprecated. Please import from diffusers.models.attention_processor instead.
  deprecate(


In [2]:
print(model_zoo)

Architectures                  Types
albef_classification           ve
albef_feature_extractor        base
albef_nlvr                     nlvr
albef_pretrain                 base
albef_retrieval                coco, flickr
albef_vqa                      vqav2
alpro_qa                       msrvtt, msvd
alpro_retrieval                msrvtt, didemo
blip_caption                   base_coco, large_coco
blip_classification            base
blip_feature_extractor         base
blip_image_text_matching       base, large
blip_nlvr                      nlvr
blip_pretrain                  base
blip_retrieval                 coco, flickr
blip_vqa                       vqav2, okvqa, aokvqa
blip2_opt                      pretrain_opt2.7b, pretrain_opt6.7b, caption_coco_opt2.7b, caption_coco_opt6.7b
blip2_t5                       pretrain_flant5xl, pretrain_flant5xl_vitL, pretrain_flant5xxl, caption_coco_flant5xl
blip2_feature_extractor        pretrain, pretrain_vitL, coco
blip2                      

In [3]:
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess, load_model

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model, vision_processer, lang_processer = load_model_and_preprocess(name="blip2", model_type="pretrain")

In [ ]:
model.to(device)

In [ ]:
vision_processer

In [ ]:
lang_processer['train'] # no use!

In [ ]:
vision_encoder = model.visual_encoder

In [ ]:
img_url = 'https://storage.googleapis.com/sfr-vision-language-research/LAVIS/assets/merlion.png' 
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')   
display(raw_image.resize((596, 437)))

In [ ]:
text = "the statue foutain in front of the lake"
text_input = [text]

In [11]:
image = vision_processer["eval"](raw_image).unsqueeze(0).to(device)
image.size()

torch.Size([1, 3, 224, 224])

In [12]:
samples = {"image": image, "text_input": text_input}

In [13]:
features = model.extract_features(samples, mode="multimodal")

In [29]:
features.multimodal_embeds

tensor([[[ 0.5422,  0.6682,  0.0352,  ..., -0.3806, -1.1119,  0.7965],
         [ 0.1159,  0.0939,  0.0707,  ...,  0.2223, -0.6508,  0.8350],
         [ 0.3911,  0.5247,  0.2072,  ..., -0.0618, -0.3508,  1.2351],
         ...,
         [ 0.1843,  0.2212, -0.0747,  ..., -0.1211, -0.5983,  0.5347],
         [ 0.6475,  0.7186, -0.0587,  ..., -0.4281, -1.3712,  0.6406],
         [ 0.4712,  0.1002,  0.3081,  ...,  0.1149, -0.6088,  0.8525]]],
       device='cuda:0')

### process inputs

In [14]:
def getAttMap(img, attMap, blur = True, overlap = True):
    attMap -= attMap.min()
    if attMap.max() > 0:
        attMap /= attMap.max()
    attMap = skimage_transform.resize(attMap, (img.shape[:2]), order = 3, mode = 'constant')
    if blur:
        attMap = filters.gaussian_filter(attMap, 0.02*max(img.shape[:2]))
        attMap -= attMap.min()
        attMap /= attMap.max()
    cmap = plt.get_cmap('jet')
    attMapV = cmap(attMap)
    attMapV = np.delete(attMapV, 3, 2)
    if overlap:
        attMap = 1*(1-attMap**0.7).reshape(attMap.shape + (1,))*img + (attMap**0.7).reshape(attMap.shape+(1,)) * attMapV
    return attMap


normalize = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))

transform = transforms.Compose([
    transforms.Resize((224,224),interpolation=Image.BICUBIC),
    transforms.ToTensor(),
    normalize,
])     

In [15]:
def pre_caption(caption,max_words=30):
    caption = re.sub(
        r"([,.'!?\"()*#:;~])",
        '',
        caption.lower(),
    ).replace('-', ' ').replace('/', ' ')

    caption = re.sub(
        r"\s{2,}",
        ' ',
        caption,
    )
    caption = caption.rstrip('\n') 
    caption = caption.strip(' ')

    #truncate caption
    caption_words = caption.split(' ')
    if len(caption_words)>max_words:
        caption = ' '.join(caption_words[:max_words])            
    return caption

In [16]:
pre_text = pre_caption(text)
pre_text

'the statue foutain in front of the lake'

In [17]:
image = transform(raw_image).unsqueeze(0)  

In [18]:
image.size()

torch.Size([1, 3, 224, 224])

In [19]:
model.tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[DEC]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [20]:
token_text = model.tokenizer(pre_text, return_tensors='pt')
token_text

{'input_ids': tensor([[ 101, 1996, 6231, 1042, 5833, 8113, 1999, 2392, 1997, 1996, 2697,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [21]:
image.to(device)

tensor([[[[ 0.5289,  0.5581,  0.5727,  ..., -0.2886, -0.2886, -0.3032],
          [ 0.5581,  0.5435,  0.5581,  ..., -0.2594, -0.3324, -0.3470],
          [ 0.5727,  0.5873,  0.5581,  ..., -0.2594, -0.3470, -0.3470],
          ...,
          [-0.5660, -0.5660, -0.5660,  ..., -0.7412, -0.7266, -0.7704],
          [-0.5660, -0.5806, -0.6390,  ..., -0.8288, -0.7850, -0.8142],
          [-0.6390, -0.5806, -0.6098,  ..., -0.8872, -0.8872, -0.9018]],

         [[ 0.6792,  0.7092,  0.7242,  ...,  0.3040,  0.2890,  0.2890],
          [ 0.6942,  0.6942,  0.7092,  ...,  0.3340,  0.2589,  0.2740],
          [ 0.7242,  0.7392,  0.7092,  ...,  0.3340,  0.2589,  0.2740],
          ...,
          [-0.7316, -0.7466, -0.7316,  ..., -0.5665, -0.5365, -0.5665],
          [-0.7616, -0.7616, -0.7766,  ..., -0.6715, -0.6115, -0.6265],
          [-0.7916, -0.7016, -0.5815,  ..., -0.7316, -0.7316, -0.7316]],

         [[ 1.0794,  1.1078,  1.1221,  ...,  1.2643,  1.2074,  1.2074],
          [ 1.0936,  1.1078,  

In [28]:
image.cuda()

tensor([[[[ 0.5289,  0.5581,  0.5727,  ..., -0.2886, -0.2886, -0.3032],
          [ 0.5581,  0.5435,  0.5581,  ..., -0.2594, -0.3324, -0.3470],
          [ 0.5727,  0.5873,  0.5581,  ..., -0.2594, -0.3470, -0.3470],
          ...,
          [-0.5660, -0.5660, -0.5660,  ..., -0.7412, -0.7266, -0.7704],
          [-0.5660, -0.5806, -0.6390,  ..., -0.8288, -0.7850, -0.8142],
          [-0.6390, -0.5806, -0.6098,  ..., -0.8872, -0.8872, -0.9018]],

         [[ 0.6792,  0.7092,  0.7242,  ...,  0.3040,  0.2890,  0.2890],
          [ 0.6942,  0.6942,  0.7092,  ...,  0.3340,  0.2589,  0.2740],
          [ 0.7242,  0.7392,  0.7092,  ...,  0.3340,  0.2589,  0.2740],
          ...,
          [-0.7316, -0.7466, -0.7316,  ..., -0.5665, -0.5365, -0.5665],
          [-0.7616, -0.7616, -0.7766,  ..., -0.6715, -0.6115, -0.6265],
          [-0.7916, -0.7016, -0.5815,  ..., -0.7316, -0.7316, -0.7316]],

         [[ 1.0794,  1.1078,  1.1221,  ...,  1.2643,  1.2074,  1.2074],
          [ 1.0936,  1.1078,  

In [22]:
token_text.to(device)

{'input_ids': tensor([[ 101, 1996, 6231, 1042, 5833, 8113, 1999, 2392, 1997, 1996, 2697,  102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

### pass through layers

In [24]:
model.Qformer.base_model.base_model.encoder.layer[8].crossattention.self.save_attention = True
model.Qformer.base_model.base_model.encoder.layer[8].crossattention.self.save_attention

True

In [29]:
samples = {'image': image, 'text_input': pre_text}

In [30]:
output = model(samples)

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [32]:
output = model(image, text_input)
loss = output[:,1].sum()

model.zero_grad()
loss.backward()    

with torch.no_grad():
    mask = text_input.attention_mask.view(text_input.attention_mask.size(0),1,-1,1,1)

    grads=model.text_encoder.base_model.base_model.encoder.layer[block_num].crossattention.self.get_attn_gradients()
    cams=model.text_encoder.base_model.base_model.encoder.layer[block_num].crossattention.self.get_attention_map()

    cams = cams[:, :, :, 1:].reshape(image.size(0), 12, -1, 24, 24) * mask
    grads = grads[:, :, :, 1:].clamp(0).reshape(image.size(0), 12, -1, 24, 24) * mask

    gradcam = cams * grads
    gradcam = gradcam[0].mean(0).cpu().detach()

AttributeError: 'Blip2Qformer' object has no attribute 'output_attentions'